In [ ]:
from __future__ import print_function

import click
import cv2
import numpy as np
import torch
from torch.autograd import Variable
from torchvision import models, transforms

from grad_cam import (BackPropagation, Deconvolution, GradCAM, GuidedBackPropagation)

In [ ]:
image_path = 'samples/cat_dog.png'
cuda = True
arch = 'resnet152'
topk = 3

CONFIG = {
    'resnet152': {
        'target_layer': 'layer4.2',
        'input_size': 224
    },
    'vgg19': {
        'target_layer': 'features.36',
        'input_size': 224
    },
    'vgg19_bn': {
        'target_layer': 'features.52',
        'input_size': 224
    },
    'inception_v3': {
        'target_layer': 'Mixed_7c',
        'input_size': 299
    },
    'densenet201': {
        'target_layer': 'features.denseblock4',
        'input_size': 224
    },
    # Add your model
}.get(arch)

In [ ]:
device = torch.device('cuda' if cuda and torch.cuda.is_available() else 'cpu')

if cuda:
    current_device = torch.cuda.current_device()
    print('Running on the GPU:', torch.cuda.get_device_name(current_device))
else:
    print('Running on the CPU')

In [ ]:
# Synset words
classes = list()
with open('samples/synset_words.txt') as lines:
    for line in lines:
        line = line.strip().split(' ', 1)[1]
        line = line.split(', ', 1)[0].replace(' ', '_')
        classes.append(line)

In [ ]:
# Model
model = models.__dict__[arch](pretrained=True)
model.to(device)
model.eval()

In [ ]:
# Image
raw_image = cv2.imread(image_path)[..., ::-1]
raw_image = cv2.resize(raw_image, (CONFIG['input_size'], ) * 2)
image = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )
])(raw_image).unsqueeze(0)

In [ ]:
def save_gradcam(filename, gcam, raw_image):
    h, w, _ = raw_image.shape
    gcam = cv2.resize(gcam.copy(), (w, h))
    gcam = cv2.applyColorMap(np.uint8(gcam * 255.0), cv2.COLORMAP_AUTUMN)
    gcam = gcam.astype(np.float) + raw_image.astype(np.float)
    gcam = gcam / gcam.max() * 255.0
    # cv2.imwrite(filename, np.uint8(gcam))
    return np.uint8(gcam)

def save_gradient(filename, data):
    data = data.copy()
    data -= data.min()
    data /= data.max()
    data *= 255.0
    # cv2.imwrite(filename, np.uint8(data))
    return np.uint8(data)

In [ ]:
gcam = GradCAM(model=model)
_, _ = gcam.forward(image.to(device))

gbp = GuidedBackPropagation(model=model)
probs, idx = gbp.forward(image.to(device))

for i in range(0, topk):
    print('[{:.5f}] {}'.format(probs[i], classes[idx[i]]))
    
    #gcam
    gcam.backward(idx=idx[i])
    region = gcam.generate(target_layer=CONFIG['target_layer'])
    gcam_image = save_gradcam(None, region, raw_image)

    #gbp
    gbp.backward(idx=idx[i])
    feature = gbp.generate()
    gbp_image = save_gradient(None, feature)

    #g-gcam
    h, w, _ = feature.shape
    region = cv2.resize(region, (w, h))[..., np.newaxis]
    output = feature * region
    ggcam_image = save_gradient(None, output)
    
    fig, axes = plt.subplots(1, 4, figsize=(15, 5))
    axes[0].imshow(raw_image)
    axes[1].imshow(gcam_image)
    axes[2].imshow(gbp_image)
    axes[3].imshow(ggcam_image)
    for ax in axes:
        ax.axis('off')